In [1]:
from haystack.nodes import PromptNode, Shaper, PromptModel
from haystack import Pipeline, Document, Answer

/home/tstad/miniconda3/envs/haystack-dev-py38/lib/python3.8/site-packages/espnet2/gan_tts/vits/vits.py:43: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(torch.__version__) >= LooseVersion("1.6.0"):


In [2]:
from haystack.nodes.prompt import PromptNode

pn = PromptNode()

pn("What is the capital of Germany?", num_return_sequences=3, num_beams=3)

/home/tstad/miniconda3/envs/haystack-dev-py38/lib/python3.8/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541702/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


{'results': ['berlin']}

In [2]:
p = Pipeline()
pm = PromptModel(model_kwargs={"model_kwargs": {"num_return_sequences": 3, "num_beams": 3}})
p.add_node(
    component=PromptNode(model_name_or_path=pm, default_prompt_template="question-answering"),
    name="PromptNode",
    inputs=["Query"],
)

In [3]:
p.run(query="What is the capital of Germany?", documents=[Document("Berlin is the capital of Germany.")])

{'results': [<Answer {'answer': 'Berlin', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['1a7644ef76698b7a1c6ed23c357fa598'], 'meta': {}}>,
  <Answer {'answer': 'Berlin is the capital of Germany.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['1a7644ef76698b7a1c6ed23c357fa598'], 'meta': {}}>,
  <Answer {'answer': 'Berlin is the capital city of Germany.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['1a7644ef76698b7a1c6ed23c357fa598'], 'meta': {}}>],
 'invocation_context': {'query': 'What is the capital of Germany?',
  'documents': [<Document: {'content': 'Berlin is the capital of Germany.', 'content_type': 'text', 'score': None, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '1a7644ef76698b7a1c6ed23c357fa598'}>],
  'results'

In [2]:
from getpass import getpass

api_key_prompt = "Enter OpenAI API key:"
api_key = getpass(api_key_prompt)

In [3]:
prompt_open_ai = PromptModel("text-davinci-003", api_key=api_key)

In [4]:
# pn = PromptNode(default_prompt_template="question-answering-with-references", model_name_or_path="google/flan-t5-base", output_variable="answers")
pn = PromptNode(
    default_prompt_template="question-answering-with-references",
    model_name_or_path=prompt_open_ai,
    shaper_params={"pattern": "Document[$idx]: $content;"},
)

TypeError: __init__() got an unexpected keyword argument 'shaper_params'

In [6]:
pn(query="Why does probability work?", documents=[Document(content="No one knows why probability works.")])

{'answers': [<Answer {'answer': 'Answering is not possible given the available information.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {}}>]}

In [6]:
p = Pipeline()
p.add_node(component=pn, name="PromptNode", inputs=["Query"])

In [4]:
documents = [
    Document(
        """Beer is the oldest[1][2][3] and most widely consumed[4] type of alcoholic drink in the world, and the third most popular drink overall after potable water and tea.[5] It is produced by the brewing and fermentation of starches, mainly derived from cereal grains—most commonly from malted barley, though wheat, maize (corn), rice, and oats are also used. During the brewing process, fermentation of the starch sugars in the wort produces ethanol and carbonation in the resulting beer.[6] Most modern beer is brewed with hops, which add bitterness and other flavours and act as a natural preservative and stabilizing agent. Other flavouring agents such as gruit, herbs, or fruits may be included or used instead of hops. In commercial brewing, the natural carbonation effect is often removed during processing and replaced with forced carbonation.[7]"""
    )
]

In [8]:
p.run(query="What is the most popular drink?", documents=documents)

{'answers': [<Answer {'answer': 'Potable water is the most popular drink, followed by tea and beer as stated in Document[5].', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {}}>],
 'invocation_context': {'query': 'What is the most popular drink?',
  'documents': [<Document: {'content': 'Beer is the oldest[1][2][3] and most widely consumed[4] type of alcoholic drink in the world, and the third most popular drink overall after potable water and tea.[5] It is produced by the brewing and fermentation of starches, mainly derived from cereal grains—most commonly from malted barley, though wheat, maize (corn), rice, and oats are also used. During the brewing process, fermentation of the starch sugars in the wort produces ethanol and carbonation in the resulting beer.[6] Most modern beer is brewed with hops, which add bitterness and other flavours and act as a natural preservative and stabilizing age

In [5]:
input_shaper_questions = Shaper(
    func="value_to_list", inputs={"value": "query"}, outputs=["questions"], params={"target_list": [1]}
)

In [6]:
input_shaper_documents = Shaper(
    func="join_documents", inputs={"documents": "documents"}, outputs=["documents"], params={"delimiter": " - "}, publish_outputs=False
)

In [7]:
output_shaper_answers = Shaper(func="strings_to_answers", inputs={"strings": "results"}, outputs=["answers"])

In [8]:
pn = PromptNode(
    default_prompt_template="question-answering",
    model_name_or_path=prompt_open_ai,
)

In [9]:
p = Pipeline()
p.add_node(component=input_shaper_documents, name="InputShaperDocuments", inputs=["Query"])
p.add_node(component=input_shaper_questions, name="InputShaperQuestions", inputs=["InputShaperDocuments"])
p.add_node(component=pn, name="PromptNode", inputs=["InputShaperQuestions"])
p.add_node(component=output_shaper_answers, name="OutputShaperAnswers", inputs=["PromptNode"])

In [10]:
res = p.run(
    query="Why does probability work?", documents=[Document(content="test"), Document(content="test2")], debug=True
)

In [11]:
res["_debug"]["PromptNode"]["runtime"]

{'prompts_used': ['Given the context please answer the question. Context: test - test2; Question: Why does probability work?; Answer:']}

In [12]:
res["documents"]

[<Document: {'content': 'test', 'content_type': 'text', 'score': None, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '9a128231f9bd4d82ac7d28cc74bde19d'}>,
 <Document: {'content': 'test2', 'content_type': 'text', 'score': None, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'd1c073b108e8511df7237f6031462f96'}>]

In [13]:
res

{'answers': [<Answer {'answer': 'Probability works because it provides a way to measure the likelihood of an event occurring based on a given set of conditions. It uses mathematical equations to calculate the chances of a certain outcome, which can be used to make predictions or analyze data.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {}}>],
 'invocation_context': {'query': 'Why does probability work?',
  'documents': ['test - test2'],
  'questions': ['Why does probability work?'],
  'results': ['Probability works because it provides a way to measure the likelihood of an event occurring based on a given set of conditions. It uses mathematical equations to calculate the chances of a certain outcome, which can be used to make predictions or analyze data.'],
  'answers': [<Answer {'answer': 'Probability works because it provides a way to measure the likelihood of an event occurring based on